## AI Server SDK for Python

### Install Dependencies

In [ ]:
%pip install --upgrade ai-server-sdk
%pip install python-dotenv

## Server Connection & Insights

### Creating the Server Connection

In [3]:
import os
from ai_server import ServerClient
from dotenv import load_dotenv

load_dotenv('../.env')
SECRET_KEY = os.getenv('DEV_SECRET_KEY')
ACCESS_KEY = os.getenv('DEV_ACCESS_KEY')

# Here we connect to the demo instance of CFG, but you would connect to your own instance
connection_url = os.getenv('DEV_ENDPOINT')

# This object creates a connection to the CFG server
server_connection = ServerClient(base = connection_url, access_key = ACCESS_KEY, secret_key = SECRET_KEY)

# Check if you are connected to the server
is_connected = server_connection.connected
print(f"Am I connected to the server? {is_connected}")

# Initilizing the server creates a new insight which is accessible through .cur_insight
my_insight = server_connection.cur_insight
print(f"This is my current insight ID: {my_insight}")

Am I connected to the server? True
This is my current insight ID: 9e32cfb7-9474-489f-81a7-d00acf4930db


### Create new insights

In [4]:
# You can create new insights by calling the .make_new_insight() method
new_insight = server_connection.make_new_insight() # This becomes your active insight

print(f"My current insight is now: {server_connection.cur_insight}")

My current insight is now: 89826093-2c63-48e5-895e-f794085ef4e3


### Listing open insights

In [5]:
my_open_insights = server_connection.get_open_insights()

print(f"These are my open insights: {my_open_insights}")

These are my open insights: ['9e32cfb7-9474-489f-81a7-d00acf4930db', '89826093-2c63-48e5-895e-f794085ef4e3']


### Dropping insights 

In [6]:
# You can drop all of your insights with the .drop_insights() method and passing in the list of insights
# Here we drop all of our open insight ids 
server_connection.drop_insights(my_open_insights) # This will create a new insight id since there must be an active insight

print(f"Here are my insight IDs after dropping: {server_connection.get_open_insights()}")

Here are my insight IDs after dropping: ['e69849f5-8956-494b-b6ce-596e7d6259c4']


### Python trick to check attributes

In [7]:
attributes = dir(server_connection)
# Find all of the attributes available to the server connection object
attributes = [attr for attr in attributes if not attr.startswith('__')]

print(f"Here are the attributes of the server connection object: {attributes}")

Here are the attributes of the server connection object: ['access_key', 'auth_headers', 'bearer_token', 'bearer_token_provider', 'connected', 'cookies', 'cur_insight', 'da_server', 'drop_insights', 'get_auth_headers', 'get_open_insights', 'get_openai_endpoint', 'get_partial_responses', 'get_pixel_output', 'import_data_product', 'loginBearerToken', 'loginUserAccessKey', 'logout', 'main_url', 'make_new_insight', 'monitors', 'open_insights', 'r', 'run_pixel', 'run_pixel_separate_thread', 'secret_key', 'send_request', 'upload_files']


## ModelEngine

### Basic Inferencing

In [8]:
from ai_server import ModelEngine

# An ID for the specific LLM you want to interact with
engine_id = os.getenv('DEV_LLM_CHAT_ENGINE_ID')

# We need to pass the engine id and an insight id to the ModelEngine object
llama3_model = ModelEngine(engine_id=engine_id, insight_id = server_connection.cur_insight)

question = "What weighs more, a pound of feathers or a pound of bricks?"

# The ask method sends a question to the model and returns the response
try:
    answer = llama3_model.ask(question)
except Exception as e:
    print(f"An error occured: {e}")

print(answer['response'])

print("We also get acess to: ")
print(f"messageId: {answer['messageId']}")
print(f"roomId: {answer['roomId']}")
print(f"numberOfTokensInPrompt", answer['numberOfTokensInPrompt'])
print(f"numberOfTokensInResponse", answer['numberOfTokensInResponse'])


Both a pound of feathers and a pound of bricks weigh one pound. The difference lies in their density and volume, not their weight.
We also get acess to: 
messageId: 8eacdf5b-ff13-48d8-b693-dff01471949d
roomId: e69849f5-8956-494b-b6ce-596e7d6259c4
numberOfTokensInPrompt 12
numberOfTokensInResponse 24


### Advanced Inferencing with parameters

In [9]:
# We can add parameters to our requests such as context, history, max_new_tokens, repetition_penalty, seed, temperature, top_k, top_p, truncate, typical_p
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are a first grade teacher that uses language and explanations easy for children to understand."
    }

# Pass your parameters as a dictionary to the ask method using the param_dict argument
new_answer = llama3_model.ask(question, param_dict=params)

print(new_answer['response'])

We already talked about this. A pound of feathers or a pound of bricks both weigh the same - ONE POUND. They are like twins, they are the same weight!


### Using chat history

In [12]:

# We can pass our own history to the ask method
# You can use as many dictionaries as you want in the history list but each one will add tokens to the request
history = [
    {"role": "user", "content": question},
    {"role": "assistant", "content": new_answer['response']}
]

# Here we change the context to a college professor and pass our chat history
params = {
    "temperature": 0.9, 
    "max_new_tokens": 200,
    "context": "You are college professor who provides complex answers backed by science.",
    "history": history
}

# Our new question references the chat history
new_question = "Can you explain your previous answer in more detail?"

new_answer = llama3_model.ask(new_question, params)

print(new_answer['response'])

Let's dive deeper into the concept of density and how it relates to mass and volume.

Density is a fundamental property of matter that describes how much mass is packed into a given volume of a substance. It's calculated by dividing the mass of an object by its volume (density = mass/volume). The density of an object is typically expressed in units of mass per unit volume, such as grams per cubic centimeter (g/cm³) or kilograms per cubic meter (kg/m³).

When comparing a pound of feathers to a pound of bricks, we can see that they have the same mass (1 pound), but their volumes are vastly different. A pound of feathers occupies a much larger volume than a pound of bricks due to its low density.

To illustrate this point, consider a liter (or a cubic decimeter, dm³) of each substance. A liter of feathers would weigh about 0.036 pounds, while a liter of lead would weigh approximately 1.06 pounds (or 1 kilogram). This means that a liter of lead has a much higher density than a liter of fea

## VectorEngine

### Adding documents

In [13]:
from ai_server import VectorEngine

# Using a FAISS vector engine
vector_engine_id = os.getenv('DEV_VECTOR_ENGINE_ID')

# We initialize the VectorEngine object with the engine id and the current insight id
faiss_vector_engine = VectorEngine(engine_id=vector_engine_id, insight_id=server_connection.cur_insight)

file_path = "./ai-whitehouse.pdf"

# We can add documents to the faiss index with the addDocument method
faiss_vector_engine.addDocument(file_paths = [file_path])

True

### Listing uploaded documents

In [14]:
# Fetch a list of uploaded documents
my_documents = faiss_vector_engine.listDocuments()

print(my_documents)

[{'fileName': 'constitution.pdf', 'fileSize': 404.2470703125, 'lastModified': '2025-03-21 16:18:35'}, {'fileName': 'ai-whitehouse.pdf', 'fileSize': 578.16015625, 'lastModified': '2025-03-25 18:07:39'}]


### Performing a nearest neighbor search

In [15]:
query = "How does the document define machine learning?"

# Find the closest match(es) between the question bassed in and the embedded documents using Euclidena Distance.
nearest_neighbor = faiss_vector_engine.nearestNeighbor(search_statement=query, limit = 3, insight_id = server_connection.cur_insight)

for index, result in enumerate(nearest_neighbor):
    print(f"Result {index + 1}")
    print(f"SCORE: {result['Score']}")
    print(f"TOKENS: {result['Tokens']}")
    print(f"CONTENT: {result['Content']}")

Result 1
SCORE: 0.7842901945114136
TOKENS: 165
CONTENT: information, such as images, videos, audio clips, and text, that has been significantly modified or generated by algorithms, including by AI. (ff ) The term “testbed” means a facility or mechanism equipped for conducting rigorous, transparent, and replicable testing of tools and 11/15/23, 10:36 AM Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence | The White House https://www.whitehouse.gov/briefing-room/presidential-actions/2023/10/30/executive-order-on-the-safe-secure-and-trustworthy-development-and-u… 10/63 technologies, including AI and PETs, to help evaluate the functionality, usability, and performance of those tools or technologies.
Result 2
SCORE: 0.7212790846824646
TOKENS: 164
CONTENT: of Artificial Intelligence | The White House https://www.whitehouse.gov/briefing-room/presidential-actions/2023/10/30/executive-order-on-the-safe-secure-and-trustworthy-development-and-us… 6

### Removing documents

In [16]:
# Names of the files we want to remove
file_names = ['ai-whitehouse.pdf']

faiss_vector_engine.removeDocument(file_names = file_names)

print(faiss_vector_engine.listDocuments())

[{'fileName': 'constitution.pdf', 'fileSize': 404.2470703125, 'lastModified': '2025-03-21 16:18:35'}]


## DatabaseEngine

### Querying a database

In [17]:
from ai_server import DatabaseEngine

# An example H2 diabetes database
db_engine_id = os.getenv('DEV_DATABASE_ENGINE_ID')

# Connect to the database by passing the engine id and the current insight id
db = DatabaseEngine(engine_id=db_engine_id, insight_id=server_connection.cur_insight)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE height < 62 AND weight > 200")

print(query)

     LOCATION HEIGHT WEIGHT
0  Buckingham     61    256
1         Bjc     60    220
2      Louisa     61    203
3      Louisa     59    204
4      Louisa     61    220
5      Louisa     58    210


### Inserting data

In [18]:
insert = "INSERT INTO diabetes (height, weight, location) VALUES (65, 200, 'Rosslyn')"

# NOTE you will need to be an author or editor of the database to insert or delete data
db.insertData(query = insert)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

  LOCATION HEIGHT WEIGHT
0  Rosslyn     65    200


### Deleting data

In [19]:
remove = "DELETE FROM diabetes WHERE location = 'Rosslyn'"

db.removeData(query = remove)

query = db.execQuery(query = "SELECT height, weight, location FROM diabetes WHERE location = 'Rosslyn'")

print(query)

None


## FunctionEngine

In [ ]:
from ai_server import FunctionEngine

# Weather function engine
weather_id = os.getenv('DEV_FUNCTION_ENGINE_ID')

function = FunctionEngine(engine_id=weather_id, insight_id=server_connection.cur_insight)

# Parameters will change based on the function you are using
output = function.execute({"lat":"37.540","lon":"77.4360"})

print(output)

## StorageEngine

### Listing Files

In [21]:
from ai_server import StorageEngine

# Example S3 bucket
storage_engine_id = os.getenv('DEV_STORAGE_ENGINE_ID')

storageEngine = StorageEngine(engine_id = storage_engine_id, insight_id = server_connection.cur_insight)

s3_storage_path = "/my-new-test-folder/"

# A list of the files in the given path
my_dir = storageEngine.list(storagePath = s3_storage_path)

for file in my_dir:
    print (f"-- {file}")

# A list of details about the files in the given path
my_dir_details = storageEngine.listDetails(storagePath = '/my-new-test-folder/')

for file_details in my_dir_details:
    print (f"-- {file_details}")


-- machine-readable-business-employment-data-mar-2024-quarter.csv
-- version/
-- {'Path': 'machine-readable-business-employment-data-mar-2024-quarter.csv', 'Name': 'machine-readable-business-employment-data-mar-2024-quarter.csv', 'Size': 3583192.0, 'MimeType': 'text/csv; charset=utf-8', 'ModTime': '2024-06-13T19:11:49.768746813Z', 'IsDir': False, 'Tier': 'STANDARD', 'Metadata': {'atime': '2024-06-13T19:11:49.768746813Z', 'btime': '2024-11-01T21:28:04Z', 'content-type': 'text/csv; charset=utf-8', 'gid': '0', 'mode': '100640', 'mtime': '2024-06-13T19:11:49.768746813Z', 'tier': 'STANDARD', 'uid': '0'}}
-- {'Path': 'version', 'Name': 'version', 'Size': 0.0, 'MimeType': 'inode/directory', 'ModTime': '2000-01-01T00:00:00.000000000Z', 'IsDir': True}


## Running Pixels

In [22]:
# You can use the active server connection to run pixels

simple_pixel_response = server_connection.run_pixel('1+1')

print('Simple Response --')
print(simple_pixel_response)

full_pixel_response = server_connection.run_pixel('1+1', full_response=True)

print('Full Response --')
print(full_pixel_response)

Simple Response --
2
Full Response --
{'insightID': 'e69849f5-8956-494b-b6ce-596e7d6259c4', 'pixelReturn': [{'pixelId': '20', 'pixelExpression': '1 + 1 ;', 'isMeta': False, 'timeToRun': 1, 'output': 2, 'operationType': ['OPERATION']}]}


### LLM Chat with pixels example

In [23]:
engine_id = os.getenv('DEV_LLM_CHAT_ENGINE_ID')

chat = server_connection.run_pixel(
    f"LLM ( engine = [ '{engine_id}' ] , command = [ '<encode>What is the capital of Connecticut</encode>' ] , paramValues = [ {{ 'max_new_tokens' : 200 , 'temperature' : 0.3 }} ] )"
)
print(chat)

{'numberOfTokensInResponse': 6, 'numberOfTokensInPrompt': 554, 'response': 'The capital of Connecticut is Hartford.', 'messageId': 'a9278cbf-1470-4c40-882c-7adf8cae9cdd', 'roomId': 'e69849f5-8956-494b-b6ce-596e7d6259c4'}
